# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

In [ ]:
# 테스트 사항 (wandb 연결해서 train loss 체크해보기)

## Import Libraries

In [2]:
!apt-get update && apt-get install -y python3-opencv

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease        
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done                      
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-opencv is already the newest version (3.2.0+dfsg-4ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 77 not upgraded.


In [3]:
!pip install sklearn

In [1]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

## Set Arguments & hyperparameters

In [2]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [3]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= './data'
NUM_CLS = 2

EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.00010880847680343138
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: driftmoon (use `wandb login --relogin` to force relogin)


True

In [13]:
wandb.init(project='optuna')

## Dataloader

#### Train & Validation Set loader

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']


## Model

In [6]:
import torch.nn.functional as F

class custom_CNN(nn.Module):
    def __init__(self, num_classes):
        super(custom_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        self.dropout1 = nn.Dropout2d(p=0.2)
        
        self.fc1 = nn.Linear(in_features=25*29*29, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=16)
        self.dropout2 = nn.Dropout2d(p=0.2)
        self.fc3 = nn.Linear(in_features=16, out_features=num_classes)
        
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
        x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        x = self.dropout1(x)
        
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        
        output = self.softmax(x)
        
        return output

In [ ]:
# Best hyperparameters: {'dropout_rate1': 0.2, 'fc2_input_dim': 64, 'fc3_input_dim': 16, 'dropout_rate2': 0.2, 'optimizer': 'Adam', 'LEARNING_RATE': 0.00010880847680343138}

## Utils
### EarlyStopper

In [7]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [14]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)
        wandb.log({"Train loss": self.train_mean_loss})

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)
        wandb.log({"Val loss": self.val_mean_loss})

### Metrics

In [9]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [15]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


#### Load model and other utils

In [16]:
# Load Model
model = custom_CNN(NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [17]:
model

custom_CNN(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 25, kernel_size=(5, 5), stride=(1, 1))
  (dropout1): Dropout2d(p=0.2, inplace=False)
  (fc1): Linear(in_features=21025, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (dropout2): Dropout2d(p=0.2, inplace=False)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

### epoch 단위 학습 진행

In [18]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')


  0% 0/50 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.73628940516048, Acc: 0.4664371772805508, F1-Macro: 0.31807511737089206


  2% 1/50 [01:04<52:54, 64.79s/it]

Epoch 0, Val loss: 1.0146169066429138, Acc: 0.5076923076923077, F1-Macro: 0.336734693877551
Epoch 1, Train loss: 0.7312337491247389, Acc: 0.47676419965576594, F1-Macro: 0.35433116921097263


  4% 2/50 [02:06<51:05, 63.86s/it]

...

Epoch 27, Val loss: 0.4900452923029661, Acc: 0.7846153846153846, F1-Macro: 0.782088122605364
Early stopping counter 8/10
Epoch 28, Train loss: 0.21401139866146776, Acc: 0.9363166953528399, F1-Macro: 0.9359493482309125


 58% 29/50 [32:30<26:51, 76.74s/it]

Epoch 28, Val loss: 0.5746703743934631, Acc: 0.7692307692307693, F1-Macro: 0.767247553115302
Early stopping counter 9/10
Epoch 29, Train loss: 0.19427081280284458, Acc: 0.9500860585197934, F1-Macro: 0.9498238519105293


 58% 29/50 [33:49<24:29, 70.00s/it]

Epoch 29, Val loss: 1.8514342904090881, Acc: 0.7846153846153846, F1-Macro: 0.7845643939393939
Early stopping counter 10/10
Early stopped


In [19]:
wandb.finish()

## Inference
### 모델 로드

In [20]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [21]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [22]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [23]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:02,  2.83s/it]

tensor([[0.3382, 0.6618],
        [0.9377, 0.0623],
        [0.8971, 0.1029],
        [0.3313, 0.6687],
        [0.7760, 0.2240],
        [0.0762, 0.9238],
        [0.5824, 0.4176],
        [0.3862, 0.6138],
        [0.9685, 0.0315],
        [0.0011, 0.9989],
        [0.7380, 0.2620],
        [0.9024, 0.0976],
        [0.4628, 0.5372],
        [0.9755, 0.0245],
        [0.9512, 0.0488],
        [0.9215, 0.0785],
        [0.3326, 0.6674],
        [0.9513, 0.0487],
        [0.7705, 0.2295],
        [0.8252, 0.1748],
        [0.9743, 0.0257],
        [0.9234, 0.0766],
        [0.8797, 0.1203],
        [0.1216, 0.8784],
        [0.9811, 0.0189],
        [0.9290, 0.0710],
        [0.9666, 0.0334],
        [0.9048, 0.0952],
        [0.0135, 0.9865],
        [0.1764, 0.8236],
        [0.4903, 0.5097],
        [0.0622, 0.9378]], device='cuda:0')


2it [00:06,  3.00s/it]

tensor([[0.7995, 0.2005],
        [0.2118, 0.7882],
        [0.9527, 0.0473],
        [0.7159, 0.2841],
        [0.8959, 0.1041],
        [0.1319, 0.8681],
        [0.4760, 0.5240],
        [0.1379, 0.8621],
        [0.0547, 0.9453],
        [0.3405, 0.6595],
        [0.7386, 0.2614],
        [0.2127, 0.7873],
        [0.7360, 0.2640],
        [0.1314, 0.8686],
        [0.4109, 0.5891],
        [0.9947, 0.0053],
        [0.9071, 0.0929],
        [0.5001, 0.4999],
        [0.2337, 0.7663],
        [0.0201, 0.9799],
        [0.8040, 0.1960],
        [0.8552, 0.1448],
        [0.2107, 0.7893],
        [0.5298, 0.4702],
        [0.2417, 0.7583],
        [0.1178, 0.8822],
        [0.9691, 0.0309],
        [0.5562, 0.4438],
        [0.0707, 0.9293],
        [0.9769, 0.0231],
        [0.9871, 0.0129],
        [0.9296, 0.0704]], device='cuda:0')


3it [00:10,  3.36s/it]

tensor([[0.0473, 0.9527],
        [0.0112, 0.9888],
        [0.6811, 0.3189],
        [0.9371, 0.0629],
        [0.0221, 0.9779],
        [0.8878, 0.1122],
        [0.8344, 0.1656],
        [0.8191, 0.1809],
        [0.9573, 0.0427],
        [0.9368, 0.0632],
        [0.1183, 0.8817],
        [0.9844, 0.0156],
        [0.9551, 0.0449],
        [0.9363, 0.0637],
        [0.3789, 0.6211],
        [0.2127, 0.7873],
        [0.2822, 0.7178],
        [0.9442, 0.0558],
        [0.8823, 0.1177],
        [0.2424, 0.7576],
        [0.4818, 0.5182],
        [0.9251, 0.0749],
        [0.6598, 0.3402],
        [0.8297, 0.1703],
        [0.7396, 0.2604],
        [0.4707, 0.5293],
        [0.7649, 0.2351],
        [0.8904, 0.1096],
        [0.5425, 0.4575],
        [0.0154, 0.9846],
        [0.9859, 0.0141],
        [0.6128, 0.3872]], device='cuda:0')


4it [00:10,  2.72s/it]

tensor([[0.1840, 0.8160],
        [0.4824, 0.5176],
        [0.5154, 0.4846],
        [0.5818, 0.4182]], device='cuda:0')


### 결과 저장

In [24]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)

In [26]:
df.to_csv('prediction.csv', index=False)

In [25]:
df[df['COVID'] == 1].shape

(40, 2)

In [27]:
df

file_name  COVID
0      0.png      1
1      1.png      0
2      2.png      0
3      3.png      1
4      4.png      0
..       ...    ...
95    95.png      0
96    96.png      1
97    97.png      1
98    98.png      0
99    99.png      0

[100 rows x 2 columns]